In [32]:
import pandas as pd
import csv
import sys
import ast
import random
import numpy as np
sys.path.append('../src/')
from functions import from_tsv_to_list
from functions import save_list_to_tsv_file
sys.path.append("../../")
from image_size import get_image_size  # source: https://github.com/scardine/image_size

In [2]:
df = pd.read_csv('../model_output/test_model_output.csv', index_col=0)
in_test = from_tsv_to_list('../../Master_gonito/test-A/in.tsv')

In [7]:
image_dir = '../../scraped_photos_final/'

# getting images size before rescale
img_old_sizes_list = []
for i in range(len(in_test)):
    img_width, img_height = get_image_size.get_image_size(
        image_dir + in_test[i]
    )
    img_old_sizes_list.append([img_width, img_height])
 
df['old_image_size'] = img_old_sizes_list

In [55]:
scaler_width, scaler_height = [], []
for i in range(len(df)):
    old_image_size_width = df['old_image_size'][i][0]
    old_image_size_height = df['old_image_size'][i][1]
    new_image_size_width = np.float(ast.literal_eval(df['new_image_size'][i])[0][0])
    new_image_size_height = np.float(ast.literal_eval(df['new_image_size'][i])[0][0])

    scaler_width.append(np.float(old_image_size_width)/np.float(new_image_size_width))
    scaler_height.append(np.float(old_image_size_height)/np.float(new_image_size_height))

In [45]:
df

,image_id,image_name,predicted_labels,predicted_boxes,new_image_size,old_image_size
0,0,[7],"[6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 6, 5, ...","[[60.70634460449219, 8.76680850982666, 69.1700...","[[70.0, 100.0]]","[1760, 7558]"
1,1,[9],"[5, 6, 5, 5, 5, 6, 5, 5, 5, 5, 6, 6, 6, 6, 5, ...","[[2.1323020458221436, 66.0712661743164, 11.634...","[[70.0, 100.0]]","[2304, 3014]"
2,2,[15],"[5, 6, 5, 6, 6, 6, 6, 6, 6, 5, 6, 6, 5, 5, 5, ...","[[48.23627471923828, 46.39752960205078, 55.637...","[[70.0, 100.0]]","[2897, 4454]"
3,3,[30],"[6, 6, 5, 6, 6, 6, 6, 5, 6, 5, 6, 5, 6, 6, 6, ...","[[46.3253059387207, 70.1567611694336, 64.69665...","[[70.0, 100.0]]","[1646, 3253]"
4,4,[36],"[5, 6, 6, 5, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[[46.49409484863281, 58.20073699951172, 55.803...","[[70.0, 100.0]]","[2741, 2485]"
5,5,[40],"[6, 5, 5, 5, 5, 5, 6, 5, 5, 6, 5, 5, 5, 5, 5, ...","[[50.773460388183594, 55.7615966796875, 61.415...","[[70.0, 100.0]]","[2907, 5620]"
6,6,[43],"[6, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, ...","[[51.879783630371094, 27.082918167114258, 68.5...","[[70.0, 100.0]]","[2761, 2446]"
7,7,[45],"[5, 6, 5, 6, 6, 6, 6, 6, 5, 6, 6, 6, 5, 5, 5, ...","[[14.422924995422363, 8.848169326782227, 22.43...","[[70.0, 100.0]]","[2012, 3776]"
8,8,[48],"[6, 6, 5, 5, 5, 6, 6, 5, 6, 5, 5, 5, 6, 6, 5, ...","[[34.201473236083984, 51.188316345214844, 51.5...","[[70.0, 100.0]]","[3225, 7448]"
9,9,[64],"[6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 6, ...","[[11.397929191589355, 73.3410415649414, 29.423...","[[70.0, 100.0]]","[1723, 7991]"


In [57]:
name_list, out_list = [], []
for i in range(len(df)):
    pred_labels = ast.literal_eval(df['predicted_labels'][i])
    pred_boxes = ast.literal_eval(df['predicted_boxes'][i])
    out_str = ''
    for ii in range(len(pred_labels)):
        label = pred_labels[ii]
        x0 = str(int(round(pred_boxes[ii][0],0)*scaler_width[i]))
        y0 = str(int(round(pred_boxes[ii][1],0)*scaler_height[i]))
        x1 = str(int(round(pred_boxes[ii][2],0)*scaler_width[i]))
        y1 = str(int(round(pred_boxes[ii][3],0)*scaler_height[i]))

        out_str = out_str + f'{label}:{x0},{y0},{x1},{y1} '
    
    out_str = out_str.strip(" ")
    out_list.append(out_str)
    name_list.append(ast.literal_eval(df['image_name'][i])[0])

In [31]:
out_path = '../geval_input/out.tsv'
save_list_to_tsv_file(out_path, out_list)